In [1]:
# Install the necessary libraries for the model and interface
!pip install -q transformers accelerate bitsandbytes gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.1 MB/s eta 0:00:00


In [2]:
import torch
from transformers import pipeline

# Load the text generation pipeline
# We use device_map="auto" to automatically use the GPU we selected in Step 1
print("Downloading and loading the model... this may take a minute.")

pipe = pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-1.5B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Model loaded successfully!


In [3]:
# --- SAFETY MEASURES ---
# This system prompt enforces the "Safety and Reliability" requirement.
SYSTEM_PROMPT = """
You are a supportive, empathetic mental health assistant.
Your goal is to listen to users and offer calming, non-judgmental support.

IMPORTANT SAFETY RULES:
1. You are NOT a doctor or a licensed therapist. Do NOT diagnose conditions or prescribe medication.
2. If a user mentions self-harm, suicide, or an emergency, you MUST immediately advise them to contact a professional or an emergency hotline (like 988 or local emergency services).
3. Keep your responses concise, warm, and encouraging.
"""

def respond(message, history):
    # We build the conversation format the model expects
    # This includes the "System" instruction first, then the user's message
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": message},
    ]

    # Generate the response
    # max_new_tokens=256 controls how long the bot talks (keeps it concise)
    outputs = pipe(
        messages,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7, # Controls creativity (0.7 is a good balance)
        top_k=50,
        top_p=0.95,
    )

    # Extract just the bot's new text from the result
    return outputs[0]["generated_text"][-1]["content"]

print("Bot logic and Safety Guardrails are ready!")

Bot logic and Safety Guardrails are ready!


In [4]:
import gradio as gr

# Create the Chat Interface
# We link the 'respond' function we created in Step 3
demo = gr.ChatInterface(
    fn=respond,
    title="Mental Health Support Bot",
    description="A safe space to share your thoughts. PLEASE NOTE: I am an AI, not a doctor. In emergencies, please call 988.",
    theme="soft", # Makes it look friendly and modern
    examples=["I feel anxious today.", "How can I calm down?", "I need help immediately."]
)

# Launch the app
print("Launching the interface...")
demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Launching the interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bb73c85b66989a882a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
